In [ ]:
#despesas estaduais entre 2014 e 2015.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

conectar_db = sqlite3.connect('Despesas.db')

cursor_db = conectar_db.cursor()

def gerarDataSet2014():
    
    arquivos = []
    for x in range(1, 13):
        try:
            nome_arq = "ReceitasEstaduais_"+"20140"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)
            
        except FileNotFoundError:
            nome_arq = "ReceitasEstaduais_"+"2014"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)

    dataset = []
    for pos in range(0, 12):
        dataset.append(arquivos[pos])
        
    return dataset

def gerarDataSet2015():
    arquivos = []
    for x in range(1, 9):
        try:
            nome_arq = "ReceitasEstaduais_"+"20150"+str(x)+".csv"
            arquivo = pd.read_csv(str(nome_arq), sep=',', encoding='ISO-8859-1', dtype='str')
            arquivos.append(arquivo)
            
        except FileNotFoundError as erro:
            print("Erro: ",erro)
            
    dataset = []
    for pos in range(0, 8):
        dataset.append(arquivos[pos])
        
    return dataset       
   
frame_2014 = []   
for pos in range(0, 12):
    df = pd.DataFrame(gerarDataSet2014()[pos])
    frame_2014.append(df)

union2014 = pd.concat(frame_2014)

frame_2015 = []
for pos in range(0, 8):
    df = pd.DataFrame(gerarDataSet2015()[pos])
    frame_2015.append(df)
    
union2015 = pd.concat(frame_2015)

union_all = pd.concat([union2014, union2015])

def tratarColunaCNPJORGAO(row):
    if len(str(row)) == 13:
        return "0"+str(row)
    elif str(row) == 'nan':
        return "-"
    else:
        return row

tratarColuna = (union_all['CNPJORGAO'].apply(tratarColunaCNPJORGAO))
union_all['CNPJORGAO'] = tratarColuna

def tratarColunaRECEITAS(row):
    if str(row) == 'nan':
        return "0"
    else:
        return format(float(row), ".15g")

tratarColuna_2 = (union_all['RECEITAPREVISTAATUALIZADA'].apply(tratarColunaRECEITAS))
union_all['RECEITAPREVISTAATUALIZADA'] = tratarColuna_2

tratarColuna_3 = (union_all['RECEITAREALIZADA'].apply(tratarColunaRECEITAS))
union_all['RECEITAREALIZADA'] = tratarColuna_3

def tratarColunaNominais(row):
    if str(row) == 'nan':
        return "-"
    else:
        return str(row)

tratarCOluna_4 = (union_all['TIPOADMINISTRACAO'].apply(tratarColunaNominais))
union_all['TIPOADMINISTRACAO'] = tratarCOluna_4
tratarCOluna_5 = (union_all['TIPOPODER'].apply(tratarColunaNominais))
union_all['TIPOPODER'] = tratarCOluna_5

def tratarColunaANOMES(row):
    ano = row[:4]
    mes = row[4:6]
    data = mes+"/"+ano
    return data
    
tratarCOluna_6 = (union_all['ANOMES'].apply(tratarColunaANOMES))
union_all['DATA'] = tratarCOluna_6

class gerarAnalises():
    
    def __init__(self, data, cnpj, natureza, recetprev, recetreal):
        
        d_data = []
        
        for i in data:
            mes = i.split("/")[0]
            ano = i.split("/")[1]
            minha_data = "1"+"/"+mes+"/"+ano
            date = datetime.strptime(minha_data, '%d/%m/%Y')
            d_data.append(date)
        
        self.cnpj = cnpj
        self.natureza = natureza
        
        v_recetprev = []
        v_recetreal = []
        
        for i in recetprev:
            v_recetprev.append(float(i))
            
        for i in recetreal:
            v_recetreal.append(float(i))
        
        dataset = {'cnpj':self.cnpj,
                   'data':d_data,
                   'natureza':self.natureza,
                   'receitaprevia':v_recetprev,
                   'receitarecetreal':v_recetreal
                   }
        df = pd.DataFrame(dataset)
        self.df = df
        
        
    def DistribuicaoReceitaMensal(self):
        dataframe = self.df
        
        imput_grps = dataframe.pivot_table(values=['receitarecetreal'], index=["data"], 
                                           aggfunc=np.sum, 
                                           dropna=False, margins_name="Total", margins=True)
        
        
        
        for pos in range(0, imput_grps['receitarecetreal'].size-1):
            data = str(imput_grps.index[pos]).split("-")[0]+"-"+str(imput_grps.index[pos]).split("-")[1]
            print(data," ",imput_grps['receitarecetreal'][pos])
        
    def ConectarBanco(self):
        dataframe = self.df
   
        cursor_db.execute("CREATE TABLE IF NOT EXISTS Despesa(\
                           id INTEGER PRIMARY KEY NOT NULL,\
                           CNPJ VARCHAR(14) NOT NULL,\
                           DATA VARCHAR(14) NOT NULL,\
                           NATUREZA VARCHAR(200) NOT NULL,\
                           RECEITAPREVISTA DOUBLE(20,2) NOT NULL,\
                           RECEITAREALIZADA DOUBLE(20,2) NOT NULL\
                          )")
        cnpj = []
        for i in dataframe['cnpj']:
            cnpj.append(i)
        data = []
        for i in dataframe['data']:
            data.append(i)
        natureza = []
        for i in dataframe['natureza']:
            natureza.append(i)
        receitaprevia = []
        for i in dataframe['receitaprevia']:
            receitaprevia.append(round(float(i),2))        
        receitarecetreal = []
        for i in dataframe['receitarecetreal']:
            receitarecetreal.append(round(float(i),2))
        
        for pos in range(0, len(cnpj)):
           
                cursor_db.execute("INSERT INTO Despesa (CNPJ, DATA, NATUREZA, RECEITAPREVISTA, RECEITAREALIZADA) VALUES (?, ?, ?, ?, ?)", (cnpj[pos],str(data[pos]),natureza[pos],receitaprevia[pos],receitarecetreal[pos]))
                conectar_db.commit()

        
        return True
           
        
        
        
obj = gerarAnalises(union_all['DATA'], union_all['CNPJORGAO'], union_all['NATUREZARECEITA'], union_all['RECEITAPREVISTAATUALIZADA'], union_all['RECEITAREALIZADA'])
obj.ConectarBanco()

In [ ]:
cursor_db.execute("SELECT * FROM Despesa")

for item in cursor_db.fetchall():
    print(item)

In [3]:
cursor_db.execute("DELETE FROM Despesa")